<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Import all libraries

In [ ]:
import numpy as np
import pandas as pd
import spacy
import pickle
import torch
import torch.nn.functional as F
from itertools import islice
from transformers import pipeline
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Search import Search
from Answer import Answer
import neuralcoref
import re
import gc
import utils

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Load data

In [13]:
file_name = 'search_dataset_111.pkl'
speeches = pd.read_pickle(file_name)

In [14]:
speeches.head()

,speech,party,stemmed_speech,year
876114,Our democracy renews itself every 2 years as M...,D,our democraci renew itself everi 2 year as mem...,111
876115,"Madam Speaker. Leader HOYER, fellow Members, a...",R,"madam speaker . leader hoyer , fellow member ,...",111
876116,"Thank you very much. Leader BOEHNER. Together,...",D,thank you veri much . leader boehner . togeth ...,111
876117,for your nomination this morning. Thank you to...,D,for your nomin thi morn . thank you to the mem...,111
876118,"Mr. Speaker, for purposes of debate only, I yi...",D,"mr. speaker , for purpos of debat onli , i yie...",111


Create search

In [15]:
search = Search(speeches=speeches)

republican speeches: 24578
democrat speeches: 30861


In [16]:
questions = ["What reforms were adopted by the 110th Congress?",
             "What do you believe about tax increases?"]

Graph construction

Generate answers to question with GPT-2

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
for question in questions:
    print("Question: {}".format(question))
    print("GPT-2 generation: ")
    generated = tokenizer.encode(question)
    context = torch.tensor([generated])
    past = None

    for i in range(100):
        output, past = model(context, past=past)
        token = torch.argmax(output[..., -1, :])

        generated += [token.tolist()]
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    print(sequence)

Question: What reforms were adopted by the 110th Congress?
GPT-2 generation: 
What reforms were adopted by the 110th Congress?

The first was the introduction of the National Defense Authorization Act of 2012 (NDAA), which authorized the Pentagon to spend $1.5 trillion over the next decade to increase the military's budget by $1.5 trillion. The second was the enactment of the National Defense Authorization Act of 2015 (NDAA), which authorized the Pentagon to spend $1.5 trillion over the next decade to increase the military's budget by $1.5 trillion.

The third was the enactment of
Question: What do you believe about tax increases?
GPT-2 generation: 
What do you believe about tax increases?

I think that the tax increases are going to be very good for the economy. I think that the tax increases are going to be very good for the middle class. I think that the middle class is going to be very happy. I think that the middle class is going to be very happy.

I think that the middle class is

Search

In [36]:
for question in questions:
  print("Question: {}".format(question))
  for party in ['R', 'D']:
    print("Party: {}".format(party))
    results = search.search(question, party, topk=3)
    for res in results['speech']:
      print(res + '\n')

Question: What reforms were adopted by the 110th Congress?
Party: R
Madam President, pursuant to section 204(a) of the 2008 budget resolution. S. Con. Res, 21, of the 110th Congress, I raise a point of order against the emergency designation in section 5(a) of the conference report.

Mr. Speaker, I am pleased to rise today in support of H. Res, 1648, a resolution that recognizes the importance of federal efforts to encourage adoption, and honors National Adoption Day and Month. As an avid adoption supporter, I believe that Congress must continue to promote the adoption of children into safe and loving homes. Through our work in 1997 as part of the Adoption and Safe Families Act, and more recently through the Fostering Connections to Success and Increasing Adoptions Act of 2008. Congress made significant advances in providing more options for children in need. Yet, far too many children, about 114.000, are waiting in foster care programs throughout our country for families to adopt them